In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob

import pandas as pd

In [3]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': 'C:\\AgodaGit\\passivbot\\'+filename}
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value
                if key == 'Total gain percentage':
                    d['tg_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain percentage':
                    d['adg_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy percentage':
                    d['bnkr_%'] = float(value.replace('%', '').strip())
                if key == 'Long':
                    d['long'] = bool(value)
                if key == 'Short':
                    d['short'] = bool(value)
                if key == 'Lowest equity/balance ratio':
                    d['ebr'] = float(value.replace('%', '').strip())

        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_329760/3000735749.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_329760/3000735749.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_329760/3000735749.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_329760/3000735749.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,s,tg_%,adg_%,bnkr_%,ebr,long,short,v
0,C:\AgodaGit\passivbot\backtests/binance\1000SH...,1000SHIBUSDT,9.1690,0.1770,72.24,0.458265,True,True,NaN
1,C:\AgodaGit\passivbot\backtests/binance\1000SH...,1000SHIBUSDT,0.7628,0.0543,100.00,0.978028,True,True,NaN
2,C:\AgodaGit\passivbot\backtests/binance\1000SH...,1000SHIBUSDT,-8.4750,-0.0636,100.00,0.631244,True,True,NaN
3,C:\AgodaGit\passivbot\backtests/binance\1000SH...,1000SHIBUSDT,8.1440,0.1560,19.14,0.209744,True,True,NaN
4,C:\AgodaGit\passivbot\backtests/binance\1000SH...,1000SHIBUSDT,9.0000,0.1070,17.07,0.194232,True,True,5.3
...,...,...,...,...,...,...,...,...,...
187,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,XTZUSDT,6.9860,0.0341,100.00,0.917915,True,NaN,NaN
188,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,XTZUSDT,12.3500,0.0701,100.00,0.852475,True,True,5.3
189,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,XTZUSDT,12.3500,0.0701,100.00,0.852475,True,True,5.3
190,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,XTZUSDT,12.1800,0.0672,100.00,0.962654,True,NaN,5.3


Now filter the dataframe

In [4]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
            "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
            "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
            "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
            "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
            "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
            "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
            "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
            "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
            "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
            "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
            "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
            "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
            "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
            "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
            "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
            "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
            "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
            "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
            "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

#allowed_symbols = ['ETHUSDT']

df = big_data[(big_data.s.isin(allowed_symbols)) &
              (big_data['bnkr_%'] > 95.0) &
              (big_data['tg_%'] > 0) &
              (big_data['v'] == '5.3') &
              (big_data['tg_%'] > 15.0)]
df

,file,s,tg_%,adg_%,bnkr_%,ebr,long,short,v
39,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,ALICEUSDT,16.34,0.0900,100.0,0.939401,True,True,5.3
40,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,ALICEUSDT,16.34,0.0900,100.0,0.939401,True,True,5.3
57,C:\AgodaGit\passivbot\backtests/binance\CHZUSD...,CHZUSDT,22.52,0.1230,100.0,0.875791,True,True,5.3
58,C:\AgodaGit\passivbot\backtests/binance\CHZUSD...,CHZUSDT,22.52,0.1230,100.0,0.875791,True,True,5.3
61,C:\AgodaGit\passivbot\backtests/binance\CRVUSD...,CRVUSDT,22.81,0.1240,100.0,0.934514,True,True,5.3
72,C:\AgodaGit\passivbot\backtests/binance\DOTUSD...,DOTUSDT,17.19,0.0957,100.0,0.847742,True,True,5.3
73,C:\AgodaGit\passivbot\backtests/binance\DOTUSD...,DOTUSDT,16.04,0.0898,100.0,0.955895,True,NaN,5.3
74,C:\AgodaGit\passivbot\backtests/binance\DOTUSD...,DOTUSDT,16.04,0.0898,100.0,0.955895,True,NaN,5.3
99,C:\AgodaGit\passivbot\backtests/binance\FTMUSD...,FTMUSDT,18.31,0.1000,100.0,0.953643,True,NaN,5.3
100,C:\AgodaGit\passivbot\backtests/binance\FTMUSD...,FTMUSDT,18.31,0.1000,100.0,0.953643,True,NaN,5.3
